## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-16-46-08 +0000,nyt,Apollo-Soyuz 50th Anniversary: A Handshake in ...,https://www.nytimes.com/2025/07/14/science/apo...
1,2025-07-15-16-45-48 +0000,bbc,Thousands of Afghans were moved to UK in secre...,https://www.bbc.com/news/articles/cvg8zy78787o
2,2025-07-15-16-45-25 +0000,bbc,Immigration status of benefit claimants publis...,https://www.bbc.com/news/articles/cdx5pw8pwg5o
3,2025-07-15-16-45-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
4,2025-07-15-16-37-16 +0000,bbc,Emmys 2025: Severance leads nominations follow...,https://www.bbc.com/news/articles/cjelpe2y3plo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,20
51,ukraine,10
79,tariff,7
172,putin,5
52,russia,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...,52
54,2025-07-15-02-08-23 +0000,nyt,How Trump Changed His Tone on Putin and the Ru...,https://www.nytimes.com/2025/07/14/us/politics...,45
40,2025-07-15-09-38-38 +0000,bbc,'Not our war' - Trump's Nato weapons deal for ...,https://www.bbc.com/news/articles/c14e2ydv4d6o,44
5,2025-07-15-16-22-56 +0000,bbc,Trump 'not encouraging killing' by asking Ukra...,https://www.bbc.com/news/articles/cvg6zg6kelro,42
62,2025-07-14-20-35-11 +0000,nyt,Trump Threatens Russia With Sanctions and Vows...,https://www.nytimes.com/2025/07/14/us/politics...,40


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,52,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...
54,25,2025-07-15-02-08-23 +0000,nyt,How Trump Changed His Tone on Putin and the Ru...,https://www.nytimes.com/2025/07/14/us/politics...
15,23,2025-07-15-14-57-21 +0000,bbc,Two dead after flash flooding in New Jersey an...,https://www.bbc.com/news/articles/c0j42xy47q7o
8,19,2025-07-15-16-08-00 +0000,wsj,"Inflation picked up in June, a potential sign ...",https://www.wsj.com/economy/inflation-hit-2-7-...
3,18,2025-07-15-16-45-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
1,14,2025-07-15-16-45-48 +0000,bbc,Thousands of Afghans were moved to UK in secre...,https://www.bbc.com/news/articles/cvg8zy78787o
6,14,2025-07-15-16-10-16 +0000,bbc,Air quality alerts issued in Canada and US as...,https://www.bbc.com/news/articles/c79q42y07wpo
9,12,2025-07-15-15-55-59 +0000,nyt,Three Aid Workers Were ‘Intentionally Killed’ ...,https://www.nytimes.com/2025/07/15/world/afric...
55,12,2025-07-15-00-57-05 +0000,bbc,Teddy bear made from fake human skin leads to ...,https://www.bbc.com/news/articles/c9dg3909w0yo
66,12,2025-07-14-20-00-07 +0000,nyt,Clashes Between Bedouin and Druse in Syria Kil...,https://www.nytimes.com/2025/07/14/world/middl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
